```
0: 3
1: 2
4: 4
6: 4
This means that there is a layer immediately inside the firewall (with range 3), a second layer immediately after that (with range 2), a third layer which begins at depth 4 (with range 4), and a fourth layer which begins at depth 6 (also with range 4). Visually, it might look like this:

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
```

In [1]:
import re

In [2]:
def parse_input(lines):
    regexp = r'^([0-9]+): ([0-9]+)$'
    current_pos = 0
    scanners = []
    for l in lines:
        m = re.match(regexp, l)
        if not m:
            continue
        g = m.groups()
        scanner_depth = int(g[0])
        scanner_range = int(g[1])
        for i in range(current_pos, scanner_depth):
            scanners.append(None)
        scanners.append((scanner_range, 0, 1))
        current_pos = scanner_depth + 1
    return scanners

In [3]:
assert parse_input([
    '0: 3',
    '1: 2',
    '4: 4',
    '6: 4',
]) == [(3, 0, 1), (2, 0, 1), None, None, (4, 0, 1), None, (4, 0, 1)]

In [4]:
def scanner_move(scanners):
    new_pos = []
    for s in scanners:
        if not s:
            new_pos.append(None)
            continue
        scanner_range, scanner_pos, scanner_direction = s
        
        # Do we change direction
        if scanner_pos == 0:
            scanner_direction = 1
        if scanner_pos == scanner_range - 1:
            scanner_direction = -1
            
        if scanner_range == 1:
            # Can't move
            new_pos.append((1, 0, 1))
            continue
        
        new_pos.append((scanner_range, scanner_pos + scanner_direction, scanner_direction))    
    return new_pos

In [5]:
assert scanner_move([(3, 0, 1), (2, 0, 1), None, None, (4, 0, 1), None, (4, 0, 1)]) == [(3, 1, 1), (2, 1, 1), None, None, (4, 1, 1), None, (4, 1, 1)]
scanner_move([(3, 1, 1), (2, 1, 1), None, None, (4, 1, 1), None, (4, 1, 1)])
scanner_move([(3, 2, 1), (2, 0, -1), None, None, (4, 2, 1), None, (4, 2, 1)])
scanner_move([(3, 1, -1), (2, 1, 1), None, None, (4, 3, 1), None, (4, 3, 1)])

[(3, 0, -1), (2, 0, -1), None, None, (4, 2, -1), None, (4, 2, -1)]

In [6]:
def play_game(scanners, delay=0, find_first=False):
    player_pos = -1
    caught = []
    
    # Wait for delay picoseconds
    for l in range(delay):
        scanners = scanner_move(scanners)
        
    for l in range(len(scanners)):
        # print(scanners)
        # Player move
        player_pos += 1
        if scanners[player_pos] and scanners[player_pos][1] == 0:
            caught.append((player_pos, scanners[player_pos][0]))
            if find_first:
                return caught
        # Scanner Move
        scanners = scanner_move(scanners)
        
    return caught
    

In [7]:
assert(play_game(parse_input([
    '0: 3',
    '1: 2',
    '4: 4',
    '6: 4',
])) == [(0, 3), (6, 4)])

In [8]:
with open('scanners.txt') as fh:
    lines  = fh.readlines()
lines = [l.strip() for l in lines]

In [9]:
result = play_game(parse_input(lines))

In [10]:
result

[(0, 4), (8, 5), (20, 6), (22, 12), (28, 8)]

In [11]:
sum([a * b for (a, b) in result])

648

In [12]:
def get_delay(scanners, skip=1, initial_delay=0):
    delay = initial_delay
    while True:
        if delay % 100 == 0:
            print('Trying delay: ', delay)
        caught = play_game(list(scanners), delay, find_first=True)
        if not caught:
            break
        # Scanner at depth one only has two positions
        delay += skip
    return delay   

In [13]:
assert get_delay(parse_input([
    '0: 3',
    '1: 2',
    '4: 4',
    '6: 4',
])) == 10

Trying delay:  0


In [14]:
get_delay(parse_input(lines))

Trying delay:  0
Trying delay:  100
Trying delay:  200
Trying delay:  300
Trying delay:  400
Trying delay:  500
Trying delay:  600
Trying delay:  700
Trying delay:  800
Trying delay:  900
Trying delay:  1000
Trying delay:  1100
Trying delay:  1200


KeyboardInterrupt: 

# Way too slow!!

In [15]:
def scanners_to_fast_scanners(scanners):
    fast_scanners = []
    idx = 0
    for s in scanners:
        if s:
            fast_scanners.append((idx, 2 * (s[0]-1)))
        idx += 1
    return fast_scanners 

In [16]:
assert scanners_to_fast_scanners(parse_input([
    '0: 3',
    '1: 2',
    '4: 4',
    '6: 4',
])) == [(0, 4), (1, 2), (4, 6), (6, 6)]

In [17]:
def get_delay_fast(scanners, skip=1, initial_delay=0, log_every=100000):
    fast_scanners = scanners_to_fast_scanners(scanners)
    delay = initial_delay
    while True:
        if delay % log_every == 0:
            print('Trying delay: ', delay)
        caught = [depth for (depth, repeat) in fast_scanners if (depth + delay) % repeat == 0] 
        if not caught:
            break
        # Scanner at depth one only has two positions
        delay += skip
    return delay

In [18]:
assert get_delay_fast(parse_input([
    '0: 3',
    '1: 2',
    '4: 4',
    '6: 4',
])) == 10

Trying delay:  0


In [19]:
get_delay_fast(parse_input(lines), 2, 22800)

Trying delay:  100000
Trying delay:  200000
Trying delay:  300000
Trying delay:  400000
Trying delay:  500000
Trying delay:  600000
Trying delay:  700000
Trying delay:  800000
Trying delay:  900000
Trying delay:  1000000
Trying delay:  1100000
Trying delay:  1200000
Trying delay:  1300000
Trying delay:  1400000
Trying delay:  1500000
Trying delay:  1600000
Trying delay:  1700000
Trying delay:  1800000
Trying delay:  1900000
Trying delay:  2000000
Trying delay:  2100000
Trying delay:  2200000
Trying delay:  2300000
Trying delay:  2400000
Trying delay:  2500000
Trying delay:  2600000
Trying delay:  2700000
Trying delay:  2800000
Trying delay:  2900000
Trying delay:  3000000
Trying delay:  3100000
Trying delay:  3200000
Trying delay:  3300000
Trying delay:  3400000
Trying delay:  3500000
Trying delay:  3600000
Trying delay:  3700000
Trying delay:  3800000
Trying delay:  3900000


3933124